In [99]:
# Import the libraries
import os
import numpy as np
import csv
import pandas as pd
import multiprocessing as mp
import shutil
import docker

docker_client = docker.from_env()

In [81]:
# List with all the filenames to search
FILNAMES = [
    "x_sobol.csv",
    "x_morris.csv",
    "x_lhs.csv",
    "y_sobol.csv",
    "y_morris.csv",
    "y_lhs.csv"
]

In [82]:
# Set the reference intrusion
DELTA_STAR = 60

#### Definition of the Objective Function
$$f(\mathbf{x} ) = \text{SEA} (\mathbf{x}) - \lambda | \delta(\mathbf{x}) - \delta^{\star}  |$$

In [83]:
# Definition of the objective function
f = lambda SEA,intrusion,lamda: SEA - np.multiply(lamda,np.abs(intrusion-DELTA_STAR))

In [84]:
# Get the root directories to the SEA and Intrusion Datasets
print(os.getcwd())

# Change the working directory
os.chdir('/home/ivanolar/Documentos')

parent_dir = os.path.dirname(os.getcwd())



/home/ivanolar/Documentos


In [85]:
# Set the intrusion and SEA storing directories
intrusion_dir = os.path.join(os.getcwd(),"data-Intrusion")
sea_dir = os.path.join(os.getcwd(),"data-SEA")

data_dir =  os.path.join(os.getcwd(),"data")
output_dir = os.path.join(os.getcwd(),"output")

# Print all the paths to ensure are well written
print(intrusion_dir,os.path.exists(intrusion_dir))
print(sea_dir,os.path.exists(sea_dir))
print(data_dir,os.path.exists(data_dir))
print(output_dir,os.path.exists(output_dir))


/home/ivanolar/Documentos/data-Intrusion True
/home/ivanolar/Documentos/data-SEA True
/home/ivanolar/Documentos/data True
/home/ivanolar/Documentos/output True


In [86]:
# Generate an array of Numpy arrays to read
input_data_files = []
for ii in range(0,3):
    curfil_1 = os.path.join(intrusion_dir ,FILNAMES[ii])
    input_data_files.append(np.loadtxt(curfil_1))
    
    
    

In [87]:
# Generate an array of Numpy arrays to read
intrusion_files_array = list()
sea_files_array = list()
for ii in range(3,6):
    curfil_1 = os.path.join(intrusion_dir ,FILNAMES[ii])
    curfil_2 = os.path.join(sea_dir ,FILNAMES[ii])
    intrusion_files_array.append(np.loadtxt(curfil_1))
    sea_files_array.append(np.loadtxt(curfil_2))
    
    

In [88]:
# Generate and array to check how the lamda parameter "affects the landscape"
num_iterations:int = 20
lamda_array = np.logspace(1.8,2.8,num=num_iterations,endpoint=False)

In [89]:
print(lamda_array)

[ 63.09573445  70.79457844  79.43282347  89.12509381 100.
 112.20184543 125.89254118 141.25375446 158.48931925 177.827941
 199.5262315  223.87211386 251.18864315 281.83829313 316.22776602
 354.81338923 398.10717055 446.68359215 501.18723363 562.34132519]


In [90]:
# Get arrays which store the different functions
mixed_function_sobol = list()
mixed_function_morris = list()
mixed_function_lhs = list()

In [101]:
# Start a loop to evaluate the "synthetic objective function"
for ii in range(num_iterations):

    # Compute the synthetic sobol
    sobol_synth = f(SEA=sea_files_array[0],intrusion=intrusion_files_array[0],lamda=lamda_array[ii])

    # Get the Morris synthetic computation
    morris_synth = f(SEA=sea_files_array[1],intrusion=intrusion_files_array[1],lamda=lamda_array[ii])

    # Get the LHS synthetic computation
    lhs_synth = f(SEA=sea_files_array[2],intrusion=intrusion_files_array[2],lamda=lamda_array[ii])
    
    # Go to the data folder and delete all the files
    for directory, dirnams , filnams in os.walk(data_dir):
        for nam in filnams:
            if nam.endswith('y_sobol.csv') or  nam.endswith('y_morris.csv') or nam.endswith('y_lhs.csv'):
                os.remove(os.path.join(data_dir,nam))

    # Save the files in the data directory
    np.savetxt(fname=os.path.join(data_dir,'y_sobol.csv'),X=sobol_synth)
    np.savetxt(fname=os.path.join(data_dir,'y_morris.csv'),X=morris_synth)
    np.savetxt(fname=os.path.join(data_dir,'y_lhs.csv'),X=lhs_synth)

    # Save a file with the lamda used
    np.savetxt(fname=os.path.join(data_dir,'lamda.csv'),X=np.array([lamda_array[ii]]))

    # Run the GSA Report
    os.system("""docker run --rm -v "$(pwd)"/output:/output -v "$(pwd)"/data:/data     ghcr.io/nikivanstein/gsareport:main -p /data/problem.json -d /data -o /output"""
)

    # Copy the data file into a new directory
    new_dir:str = os.path.join(os.getcwd(),f"data_{str(ii+1)}")
    if os.path.exists(new_dir):
        shutil.rmtree(new_dir,ignore_errors=True)

    shutil.copytree(data_dir,new_dir)

    # Copy the data file into a new directory
    new_dir:str = os.path.join(os.getcwd(),f"output_{str(ii+1)}")
    if os.path.exists(new_dir):
        shutil.rmtree(new_dir,ignore_errors=True)

    shutil.copytree(output_dir,new_dir)

    # Delete all the files in the output directory

    # Go to the data folder and delete all the files
    for directory, dirnams, filnams in os.walk(output_dir):
        for nam in filnams:
            if nam.endswith(".html"):
                os.remove(os.path.join(output_dir,nam))
        
    

  0%|          | 0/100 [00:00<?, ?it/s]

loaded problem definition


  0%|          | 0/100 [00:00<?, ?it/s]

loaded problem definition


100%|██████████| 100/100 [00:05<00:00, 16.91it/s]


...

  0%|          | 0/100 [00:00<?, ?it/s]

loaded problem definition


100%|██████████| 100/100 [00:05<00:00, 17.57it/s]


loaded problem definition


100%|██████████| 100/100 [00:05<00:00, 17.57it/s]
